# Configuración de cuenta y montaje de Google Storage

**Atención:** Esto es un entorno preparado para ser ejecutado desde Google Colab. Necesitas acceso al proyecto de GCP denominado UCM-GCP asociada al proyecto de investigación CABAHLA.

En primer lugar nos autenticamos con nuestra cuenta ucm.es:

In [ ]:
from google.colab import auth
auth.authenticate_user()

Ahora instalamos gcsfuse

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  27500      0 --:--:-- --:--:-- --:--:-- 27500
OK
33 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.41.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


Creamos un directorio donde vamos a montar el bucket de Google Storage:

In [ ]:
!mkdir gcs
!gcsfuse --implicit-dirs cabahla gcs

mkdir: cannot create directory ‘gcs’: File exists
2023/02/08 17:24:46.347122 Start gcsfuse/0.41.12 (Go version go1.18.4) for app "" using mount point: /content/gcs
2023/02/08 17:24:46.405510 Opening GCS connection...
2023/02/08 17:24:47.527748 Mounting file system "cabahla"...
2023/02/08 17:24:47.528679 File system has been successfully mounted.


Nos descargamos el código fuente de caide si no lo hemos hecho ya:

In [9]:
!git clone https://github.com/jlrisco/caide.git

Cloning into 'caide'...
remote: Enumerating objects: 632, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 632 (delta 20), reused 39 (delta 16), pack-reused 581
Receiving objects: 100% (632/632), 140.95 MiB | 22.97 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Updating files: 100% (125/125), done.


Finalmente, instalamos los paquetes python necesarios:

In [11]:
!pip3 install xdevs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Use case 1: Oahu

First, we create the model:

In [12]:
import sys
sys.path.insert(0, 'caide')
!python3 caide/main_test.py

Traceback (most recent call last):
  File "caide/main_test.py", line 56, in <module>
    coupled = OahuTest(model_name, os.path.join('data', 'input', 'simulations', 'main_test.txt'))
  File "caide/main_test.py", line 34, in __init__
    sensor = VirtualNode(name=sensor_name, sensors_folder=os.path.join('data','input',self.name, farm_name))
  File "/content/caide/edge.py", line 29, in __init__
    for file_entry in os.listdir(name_sensors_folder):
FileNotFoundError: [Errno 2] No such file or directory: 'data/input/DataCenter/Oahu/ap1'
